# Tokenizer deep-dive 🧐
*(contact: arjo@stanford.edu)*

The Tokenizer pulls together all we've learned so far and builds our vocabulary.

Let's dive into every step while building our vocabulary and what it does ..!

## Setup dataset for tokenization
Let's start by choosing a couple of languages and setting up a dataset we can train our tokenizer on.

In [ ]:
from data import load_wikiann_corpus
# --- 1. Define and Load the Corpus ---
# Define the languages we want to train on.
# We map short codes (e.g., 'en') to their full names for clarity.
lang_codes = { 
    'en': 'English', 'da': 'Danish',
    #'zh': 'Chinese',# 'fr': 'French',
    #'tr': 'Turkish', 'ru': 'Russian', 'ja': 'Japanese',
    #'ar': 'Arabic', 'ta': 'Tamil', 'xh': 'Xhosa',
    #'zu': 'Zulu', 'tk': 'Turkmen'
}

# Load the training data from the Hugging Face 'wikiann' dataset.
num_paragraphs = 700
corpus_texts, corpus_langs = load_wikiann_corpus(lang_codes, per_lang=num_paragraphs)
print(f"Number of sentences: {len(corpus_texts)} across {len(lang_codes)} languages")
print(f"Let's print a few examples:")
for i in range(len(lang_codes)):
    txt = corpus_texts[10+i*num_paragraphs]
    lang = corpus_langs[10+i*num_paragraphs]
    print(f"\tLanguage: {lang}, Text: {txt}")

---
Now we have an small interesting multi-lingual dataset.

---

### ScalableTokenizer.__init__(...)
This is the args description
```
Args:
    max_token_len (int): The maximum length in characters for any learned token.
    min_freq (int): A candidate token must appear at least this many times
        in the corpus to be considered for the vocabulary.
    alpha (float): A weighting factor for the token's negative log-likelihood cost.
    beta (float): A weighting factor for the token's PMI-based cohesion penalty.
    tau (float): A weighting factor for the token's length penalty.
    top_k_add (int): The number of best new tokens to add to the vocabulary
        in each training iteration.
    vocab_budget (int | None): The target size for the final vocabulary. If None,
        no budget is enforced.
    lambda_lo (float): The lower bound for the Lagrangian multiplier used
        in vocabulary budgeting.
    lambda_hi (float): The upper bound for the Lagrangian multiplier.
```
There's two groups of variables. Once related to hard constraints on the vocabulary (`max_token_len`, `min_freq`, `vocab_budget`) and hyperparameters (`alpha`, `beta`, `tau`, `lambda_lo`, `lambda_hi` `top_k_add`). For now, we don't modify most of the hyperparameters, this is something you can look into as the default values are not optimized.

In [ ]:
from tokenizer import ScalableTokenizer
# --- 2. Initialize and Configure the Tokenizer ---
# Create an instance of our ScalableTokenizer.
# We configure its core behavior with these parameters:
tokenizer = ScalableTokenizer(
    max_token_len=12,   # The longest possible token (in characters)
    min_freq=7,         # A word must appear at least 7 times in the corpus
    top_k_add=8,        # In each training step, add the 8 "best" new tokens to the vocabulary
    vocab_budget=500    # The target size for our final vocabulary (number unique tokens)
)

---
Great! Now we have our tokenizer.
However, for design purposes we have not yet filled in any linguistic information.
Thus, it is not "linguisticly-aware" yet. We do that after initializing our tokenizer using the `ScalableTokenizer.set_feature_models` function. This will initialize a `LinguisticModels` object that will be used for linguistic awareness during tokenization.

---

## LinguisticModels
This is the cores of linguistic awareness in our tokenizer. It uses linguistic signals such as lexical, sequential, morphological, and syntactic information to define a cost for each token. This can then be used downstream to pick the most valuable tokens out (i.e. see `ScalableTokenizer._dp_decode`).

Examples include:
- Keep `"New York"` as one token.
- Sequence of capitalized words might be a name, e.g. `"Jane Doe"`
- Morphologically consistant: e.g. `"walking"` consists of `"walk"` and `"-ing"`, so it will be scored higher as a full token instead of splitting it up.

## LinguisticModels.init(...)
This is the args description
```
"""
Attributes:
    lexicon (dict): A dictionary mapping known tokens to a score (reward).
    mwe (set): A set of known multi-word expressions.
    ne_gaz (dict): A named entity gazetteer, mapping entity types (e.g., "LOC")
        to sets of known entities.
    token_bigram (dict): A dictionary defining costs for transitions between
        token classes (e.g., from "InitCap" to "InitCap").
    lm_token_prob (callable | dict): An external language model providing token
        probabilities.
    paragraph_lang (callable): A function that returns the language for a given
        paragraph index.
    gamma_boundary (float): A penalty for changing token classes, which
        encourages sequences of similar token types.
    mu_morph (float): The weight applied to the score from the morphology
        encoder.
    rho_group (float): A small reward (negative cost) for tokens that contain
        known prefixes or suffixes.
    morph_encoder (MorphologyEncoder | None): An instance of the morphology
        encoder, which is trained to score the morphological "fit" of a
        token for a given language.
"""
```
The initialization allows us to manually bias our tokenizer towards certain token constructs.
Let's test this by setting `lexicon`, `mwe`, and `token_bigram`.
We skip `lm_token_prob`, `paragraph_lang` is handled later on, `gamma_boundary`, `mu_morph`, & `rho_group` are hyperparameters, and `morph_encoder` is created later on in `tokenizer._initialize_stats_and_vocab`.

In [ ]:
# --- 3. Set up Linguistic "Hints" ---
# We can give the tokenizer extra knowledge to improve its accuracy.
    
# A 'lexicon' of known multi-word phrases. The numbers are scores; higher is better.
# This encourages "New York" to be one token instead of two ("New", "York").
lex = {"New York": 2.0, "San Jose": 1.0, "’s": 0.5, "'s": 0.5}

# A 'named entity' (ne) gazetteer. This lists known entities.
# We're telling it that "New York", "Berlin", and "東京" are locations (LOC).   
ne_gaz  = {"LOC": {"New York", "Berlin", "東京"}}

# 'Token bigrams' (tb) define costs for sequences of token types.
# A negative cost is a "reward". This encourages sentences to start with a
# capitalized word and for capitalized words to follow each other (like in a name).
token_bigram  = {
    ("<BOS>", "InitCap"): -0.2,
    ("InitCap", "InitCap"): -0.3,
    ("NUM", "NUM"): -0.15,
}                   
                    
# Now, we apply these linguistic models and tune some internal algorithm weights.
tokenizer.set_feature_models(
    lexicon=lex,
    ne_gaz=ne_gaz,
    token_bigram=token_bigram,
    gamma_boundary=0.06,
    mu_morph=0.25,
    rho_group=0.06
)

---

## ScalableTokenizer.train(...)
This is where the magic happens, all the functions we've built so far gets pulled together and utilized to build our vocabulary.
Below we will spell out every step.

Importantly, this is when we provide the text, language-id per sample, and how many iterations we want to run it for.

## ScalableTokenizer._initialize_stats_and_vocab
This function is the "data ingestion and analysis" phase of the tokenizer. Its primary goal is to find every potential token in the corpus and assign it two fundamental statistical scores: one measuring its rarity and another measuring its internal cohesion.

The process can be broken down into six main steps:

#### 1. Substring Enumeration and Counting

First, the function performs a brute-force scan of the entire corpus. It slides a window of every possible length (from 1 up to `max_token_len`) across the text and counts the occurrences of every single "legal" substring. A substring is legal if it respects Unicode boundaries and doesn't improperly split protected entities like URLs (as determined by `utils.is_legal_span`).

This step produces two key objects:

* `char_count`: A simple frequency count of every individual character.
* `substr_count`: A frequency count of every valid substring found in the text.

NOTE: this can get expensive memory-wise for large corpora - how can we resolve that? ⭐

---

#### 2. Candidate Filtering

The raw `substr_count` contains a massive number of substrings, many of which are junk or noise. This step acts as a filter, creating a cleaner set of `_potential_tokens` by applying several heuristic rules:

* **Frequency:** Must appear at least `min_freq` times.
* **Noise:** Filters out tokens that are likely wiki-formatting (`''`, `==`), redirect commands, or just punctuation.
* **Structure:** Filters out tokens with too many internal spaces or strange mixtures of scripts.

This dramatically reduces the number of candidates the algorithm needs to consider.

---

#### 3. Indexing Occurrences

For speed, the algorithm needs to be able to quickly find every location a potential token appears. This step builds an index (a dictionary called `_token_occurrences`) that maps each potential token to a list of all its starting positions in the corpus.

---

#### 4. Calculating Statistical Costs (The Math) 🧠

This is the heart of the function. For every potential token, we calculate two scores that form its "base cost."

**Token Rarity: The Negative Log-Likelihood Cost (`_nll`)**

This score measures how rare a token is, conditioned on its length. The core idea is that more frequent tokens should have a lower cost. We use the negative log-likelihood, which is a standard way to turn probabilities into costs in machine learning.

First, we calculate the smoothed probability of a token t of length L:
$$P(t|L)=\frac{\texttt{count}(t) + \alpha_{\texttt{len}}}{\left(\sum_{t'\; \text{where}\; \texttt{len}(t')=L}\;\texttt{count}(t')\right) + \alpha_{\texttt{len}}\cdot N_L} $$

Where:
* $\texttt{count}(t)$ is the frequency of our specific token $t$.
* The sum in the denominator is the total count of all tokens that have the same length L.
* $\alpha_{\texttt{len}}$ is a smoothing constant (additive or Laplace smoothing) to prevent probabilities of zero for unseen tokens.
* $N_L$ is the number of unique token types of length L.

The final cost is the negative log of this probability. Tokens that are common for their length will have a high probability and thus a low cost.
$$ \text{cost}_{\text{NLL}}(t) = −\log(P(t∣L))$$

**Token Cohesion: The PMI-like Penalty (`_pmi_pen`)**

This score measures how "gluey" or cohesive a token's characters are. It answers the question: "Do these characters appear together more often than we'd expect by chance?" A high cohesion score suggests the string is a meaningful unit (like "ing") rather than a random sequence of letters.

This is based on **Pointwise Mutual Information (PMI)**. The formula is:
$$\text{PMI-score}(t) =\log\left(\frac{P(t|L)}{\prod^L_{i=1}P(c_i)}\right) $$
Where:
* $P(t∣L)$ is the length-conditioned probability we just calculated above.
* $P(c_i)$ is the probability of the i-th character in the token, derived from the initial `char_count`.
* The product $\prod P(c_i)$ represents the probability of seeing that sequence of characters if they were all statistically independent.

If the score is high, it means the numerator is much larger than the denominator—the token appears far more frequently than chance would suggest, making it a cohesive unit. The final `_pmi_pen` is the **negative** of this score, so that highly cohesive tokens get a **cost reduction (a reward)**.

---

#### 5. Initializing the Vocabulary

The tokenizer needs a starting vocabulary. This step simply initializes the vocabulary with all single characters seen in the corpus. This provides a crucial fallback: if no multi-character tokens can be formed, the system can always represent the text as a sequence of individual characters.

NOTE: How would this vocabulary handle large amounts of unique unicode symbols? ⭐

---

#### 6. Training the Morphology Encoder

Finally, using the collected statistics (`_potential_tokens` and `_token_occurrences`), the function trains the `MorphologyEncoder`. This component learns the sub-word patterns (common prefixes, suffixes, and character n-grams) of each language. Its math is self-contained (involving matrix factorization of a PPMI matrix), but it relies on the data prepared in the preceding steps to do its job.

## MorphologyEncoder.fit

The purpose of this function is to learn the morphological "shape" of words. It does this by representing tokens based on their sub-word features (character n-grams and affixes) and then using matrix factorization to discover the most important patterns. The final output is a low-dimensional vector for each token, where tokens with similar morphological patterns (like "running" and "swimming") are close to each other in the vector space.

The process unfolds in six key steps:

---

#### 1. Determine Primary Language for Each Token

A token (especially a name or a loanword) might appear in texts from multiple languages. This initial step simply assigns each token to a single, primary language based on where it appears most frequently. This ensures that a token like "start" is analyzed using English morphology rules, even if it appears once in a German text.

---

#### 2. Build the Feature Matrix $X$

This is the foundational step where we convert raw text into a numerical matrix. The matrix X represents the relationship between tokens and their morphological features.

* **Rows:** Each row corresponds to a unique token t from the corpus.
* **Columns:** Each column corresponds to a unique feature f. Features are things like character bigrams (`'th'`, `'ng'`), trigrams (`'ing'`), and known affixes (`'$suf:ing'`).
* **Values:** The cell $X_{tf}$ contains an integer count of how many times feature $f$ appears in token $t$.

For example, for the token `'running'`, the row would have non-zero values in the columns corresponding to features like `'ru'`, `'un'`, `'nn'`, `'ni'`, `'in'`, `'ng'`, `'run'`, `'unn'`, `'nni'`, `'nin'`, `'ing'`, and `'$suf:ing'`.

---

#### 3. Calculate the Positive Pointwise Mutual Information (PPMI) Matrix

Using raw counts ($X$) is not ideal because common features (like the character `'e'`) aren't very informative. We want to find features that are surprisingly common for a given token. This is what **Pointwise Mutual Information (PMI)** measures.

The formula for PMI between a token t and a feature f is:
$$\text{PMI}(t,f)=\log_2\left(\frac{P(t,f)}{P(t)P(f)}\right)$$

Let's break this down:

* $P(t,f)$: The joint probability of seeing the token $t$ and the feature $f$ together. In the code, this is calculated as `P_xy`, where each cell is the count from $X$ divided by the total sum of all counts in $X$.
* $P(t)$: The marginal probability of seeing token $t$. This is the sum of row $t$ in $X$ divided by the total sum (`P_x`).
* $P(f)$: The marginal probability of seeing feature $f$. This is the sum of column $f$ in $X$ divided by the total sum (`P_y`).

The ratio $\frac{P(t,f)}{P(t)P(f)}$ tells us how much more likely the feature and token are to co-occur than if they were independent. A large ratio means the feature is highly indicative of that token.

The code then computes **Positive PMI (PPMI)** by taking `max(0, PMI)`. This discards negative associations (where features co-occur less than chance), as they are often noisy and not useful for this task. The result is a matrix `PPMI` of the same shape as $X$, but with values that represent feature informativeness rather than raw counts.

---

#### 4. Factorize the PPMI Matrix to Get Vectors (The Math)

We now have a large, sparse `PPMI` matrix. Our goal is to create short, dense vectors (embeddings) for each token. This is a dimensionality reduction problem. The code uses **Eigendecomposition**, a powerful technique from linear algebra.

* **Create a Token-Token Similarity Matrix ($G$):** The code first computes $G=PPMI\times PPMI^T$, $G=(G+G^T)\cdot 0.5$. This results in a square matrix where each cell $G_{ij}$ represents the similarity between token $i$ and token $j$. If two tokens share many of the same informative features, their corresponding value in $G$ will be high. This is known as a Gram matrix.
* **Eigendecomposition:** The code then performs eigendecomposition on G. This breaks the matrix down into its constituent eigenvalues and eigenvectors: $G=Q\Lambda Q^{−1}$
* **Eigenvectors ($Q$):** These are the principal components of the matrix. They represent the fundamental directions of variation in the data—in this case, the most important axes of morphological similarity.
* **Eigenvalues ($\Lambda$):** These are scalar values that indicate how much variance is explained by each eigenvector. A large eigenvalue means its corresponding eigenvector is very important.
* **Construct the Embedding Matrix ($V$):** We don't need all the eigenvectors, only the most important ones. The code selects the top `k` eigenvectors corresponding to the `k` largest eigenvalues. The final token embedding for each token is constructed by taking its corresponding row from the top `k` eigenvectors, and scaling each component by the square root of its associated eigenvalue. This is a standard method that yields embeddings where the squared distance between vectors approximates the similarity relationship in the original data.
* **Normalization:** Finally, each vector in $V$ is normalized to have a unit length of 1. This is crucial because it allows us to use the dot product as a direct measure of **cosine similarity**. When comparing two unit vectors, their dot product is exactly the cosine of the angle between them, providing a pure measure of similarity irrespective of vector magnitude.

---

#### 5. Store Learned Vectors and Language Prototypes

The result of the factorization is stored:
* `token_vec`: A dictionary mapping each token string to its learned dense vector (a row from $V$).
* `lang_proto`: A dictionary mapping each language to a "prototype" vector. This prototype is calculated by **averaging the vectors of all tokens** belonging to that language. It represents the morphological center-of-gravity for that language in the learned vector space.

---

#### 6. Pre-calculate Counts for Consistency Bonus

This is a final preparatory step. It iterates through the tokens and counts, for each cross-lingual morphological category (like `PLUR` for plural), how many different languages have tokens belonging to that category. This pre-computation makes the `consistency_bonus` calculation during the main tokenization process much faster.


---

Phew that was a lot ...! let's code it up, it's just one LOC (should take ~2 mins)

In [ ]:
tokenizer._initialize_stats_and_vocab(corpus_texts, corpus_langs)

### ScalableTokenizer.train - Loop phase
The training loop continuously alternates between two main phases: the **Primal Step** (solving the problem with the current vocabulary) and the **Pricing Step** (finding new, better vocabulary to add).

---

**Phase 1: The Primal Step - Finding the Best Path with `_dp_decode` 🗺️**

In each iteration, the first thing we must do is find the best possible way to tokenize the entire corpus using our **current** vocabulary. This is what `_dp_decode` does.

The function treats tokenization as finding the "shortest path" through the text. Imagine the text as a graph where each character is a node. A potential token is an arc or "road" connecting the character node at its start to the node at its end. Each road has a toll, which is its cost. The goal is to find the sequence of roads from the beginning to the end with the lowest total toll.

This is a classic problem solved efficiently using **Dynamic Programming**, specifically the **Viterbi algorithm**.

**The Logic and the Math**

The algorithm builds a table, `dp`, where $dp[t,k]$ stores the minimum possible cost to segment the text up to character position $t$, with the very last token belonging to class $k$ (e.g., "lower", "InitCap", etc.).

It calculates this by iterating through each position $t$ in the text and looking backward for all possible split points $i$. The core recurrence relation is:
$$dp[t,j]=\min_{i,k}(dp[i,k]+\text{cost}(\text{text}[i:t]))$$

Where:
* `text[i:t]` is the new token candidate.
* $j$ is the class of this new token.
* We find the minimum cost by trying all possible previous split points $i$ and all previous token classes $k$.
* The `cost` term is a combination of the token's base statistical cost (`_base_token_cost`) and all other linguistic feature costs (`additive_cost`).

The most important output of this step is the array `dp_min`. This array, `all_dp_min` in the `train` function, stores the minimum cost to reach **every character position** in the text. $dp_{\text{min}}[t]$ is the cost of the optimal tokenization from the start of the string up to character $t$. This array is the crucial input for the next phase.

---

**Phase 2: The Pricing Step - Finding Shortcuts with `_find_best_new_tokens_batch` 💰**

Now that we know the cost of the "best path" using our current roads (vocabulary), we can search for potential shortcuts. This is the "pricing" or "column generation" step. We evaluate every potential token that is **not** currently in our vocabulary and calculate its **Reduced Cost**.

The Reduced Cost is the "profitability" of adding a new token. It answers the question: "By how much would the total tokenization cost decrease if we added this new token to our vocabulary?"

**The Logic and the Math**

For a candidate token tok that spans from start to end in the text, we can calculate the potential savings.

1. **Cost of the Old Path:** The cost to get from the beginning to position `end` using the current vocabulary is already known: it's $dp_{\text{min}}[end]$.
2. **Cost of the New Path:** If we were allowed to use our new token `tok`, the cost to get to `end` would be the cost of the best path to `start`, plus the cost of our new token itself. This is: $dp_{\text{min}}[\text{start}]+\text{cost}(\text{tok})$.

The **Reduced Cost (RC)** is the difference between these two:
$$\text{RC}(\text{tok})=(dp_{\text{min}}[\text{start}]+\text{cost}(\text{tok}))−dp_{\text{min}}[\text{end}]$$

If the Reduced Cost is **negative**, it means the new path is cheaper than the old path. This token is a valuable "shortcut" that reduces the total segmentation cost.

The `_find_best_new_tokens_batch` function iterates through every occurrence of every potential token, calculates its RC, and sums up the total potential savings for each unique token across the entire corpus. It then sorts them and returns the `top_k` tokens with the most negative total RC—these are the most valuable new "expressways" to build.

---

**Putting It Together: The Loop**

The `train` function simply repeats these two steps:
1. `_dp_decode` **(Primal):** Calculate the optimal paths and costs (`dp_min`) for the whole corpus with the current vocabulary.
2. `_find_best_new_tokens_batch` **(Pricing):** Use the `dp_min` costs to find the new tokens with the most negative Reduced Cost.
3. **Update:** Add these new, high-value tokens to the vocabulary.
4. **Repeat:** Continue this cycle until no new tokens with a significantly negative RC can be found, at which point the vocabulary has converged to an optimal state.

---

Now, lets get the vocabulary! Run the code below, should take ~10 minutes

In [ ]:
import unicodedata as ud
paragraphs_texts=corpus_texts; paragraphs_langs=corpus_langs; max_iterations=300; rc_stop_tol=-1e-6; verbose=True;

if verbose: print("\nStep 2: Starting training with batch pricing...")
for it in range(1, max_iterations + 1):
    tokenizer._cost_cache.clear()
    # Primal step: Decode the whole corpus with the current vocabulary.
    all_dp_min = []
    for pi in range(len(paragraphs_texts)):
        _, _, dpmin = tokenizer._dp_decode(pi, decode_only=False)
        all_dp_min.append(dpmin)

    # Pricing step: Find the best new tokens to add.
    new_tokens, topk_info = tokenizer._find_best_new_tokens_batch(all_dp_min, top_k=tokenizer.top_k_add)
    if not new_tokens:
        min_rc = topk_info[0][0] if topk_info else 0.0
        if verbose:
            print(f"\nConvergence: no negative reduced-cost tokens (min summed RC = {min_rc:.6f}).")
        break
    min_rc = topk_info[0][0] if topk_info else 0.0

    # Check for convergence.
    if min_rc >= rc_stop_tol:
        if verbose:
            print(f"\nConvergence: min summed RC = {min_rc:.6f} >= tol {rc_stop_tol:.1e}.")
        break

    # Add the new tokens to the vocabulary.
    for tok in new_tokens:
        tok = ud.normalize("NFC", tok)
        if tok not in tokenizer.tok2id:
            tokenizer.tok2id[tok] = len(tokenizer.vocab)
            tokenizer.vocab.append(tok)

    if verbose:
        preview = ", ".join([f"'{t}'" for t in new_tokens[:6]])
        more = "" if len(new_tokens) <= 6 else f" ... (+{len(new_tokens)-6})"
        print(f"Iter {it:02d}: Added {len(new_tokens)} tokens: {preview}{more} (best summed RC={min_rc:.4f})")
else:
    if verbose: print("\nMax iterations reached.")

# If a vocab budget is set, enforce it now.
if tokenizer.vocab_budget is not None:
    tokenizer._enforce_vocab_budget_bisection(paragraphs_texts, tokenizer.vocab_budget, verbose=verbose)
    # Prune exported vocab to active multi-char tokens (plus alphabet)
    tokenizer._prune_vocab_to_active(paragraphs_texts)

if verbose:
    print(f"\nTraining complete. Final vocabulary size: {len(tokenizer.vocab)}")

## ScalableTokenizer.tokenize

In [ ]:
for i in range(len(lang_codes)):
    lang = corpus_langs[i*num_paragraphs]
    print(f"Language: {lang}")
    for j in range(10):
        txt = corpus_texts[j+i*num_paragraphs]
        tokenized = tokenizer.tokenize(txt, lang)
        print(f"\t tokenized: {tokenized}")

In [ ]:
tokenizer.vocab[-500:]